Estado Actual del Dataset MotoFit
Hasta ahora, el dataset MotoFit ha sido consolidado y limpiado a un nivel óptimo para iniciar un Análisis Exploratorio de Datos (EDA) consistente y provechoso.

Principales logros y características del dataset:

Se han incorporado datos de todas las marcas relevantes, con más de 590 registros de modelos de motos.

Las motos han sido clasificadas cuidadosamente por tipo, utilizando la columna TIPO_CORREGIDO, basada en información validada y actualizada de cada modelo. Se han estandarizado el tipo de carnet, obteniendo el carnet mínimo para cada modelo de moto.

Se ha eliminado la columna TIPO original para evitar confusiones, quedando un solo campo para la categoría real de cada moto.

Se han manejado y reducido significativamente los valores faltantes, garantizando la calidad de las variables numéricas y categóricas clave, como potencia, cilindrada, precio, altura del asiento, entre otras. Están pendiente os valores de las ruedas y frenos porque aún no he decidido si aportarán valor relevante al análisi y procesamiento. En cambio los valores de potencia , peso, precio y altura faltantes se han comprobado y dejado como NaN para no alterar los datos disponibles y validados.

El dataset está estructurado con 14 variables limpias y relevantes para análisis, incluyendo características técnicas, requerimientos de carnet, precios y tipos de motos.

Se dispone ahora de un respaldo seguro del dataset limpio (motofit_dataset_working.csv) para asegurar la integridad del trabajo y facilitar futuros análisis.

Con este estado, el dataset está preparado para avanzar hacia un EDA avanzado, que permita descubrir patrones, diferencias entre marcas y tipos, relaciones entre variables y posibles insights para aplicaciones de análisis de mercado o recomendaciones.

In [2]:
import pandas as pd

df = pd.read_csv('motofit_dataset_working.csv')
df['PRECIO'] = pd.to_numeric(df['PRECIO'], errors='coerce')
cantidad_por_tipo = df['TIPO_CORREGIDO'].value_counts().sort_values(ascending=False)
precio_promedio_por_tipo = df.groupby('TIPO_CORREGIDO')['PRECIO'].mean().sort_values(ascending=False)

# Mostrar resultados
print("=== Cantidad de motos por tipo ===")
print(cantidad_por_tipo)

print("\n=== Precio promedio por tipo de moto ===")
print(precio_promedio_por_tipo.round(0))




=== Cantidad de motos por tipo ===
TIPO_CORREGIDO
Scooter              153
Naked                136
Adventure             87
Cruiser               69
Sport                 50
Tourer                35
Scrambler             16
Sport Tourer           9
Off-road               9
Supermoto              7
Classic                5
Adventure Scooter      4
Electric               3
MiniMOTO               3
Hyper-Naked            3
Cafe Racer             3
Bobber                 3
Adventure Tourer       2
Otro                   1
Name: count, dtype: int64

=== Precio promedio por tipo de moto ===
TIPO_CORREGIDO
Otro                 37290.0
Off-road             25094.0
Hyper-Naked          23250.0
Tourer               21741.0
Sport Tourer         20534.0
Cruiser              18868.0
Sport                17500.0
Electric             17273.0
Supermoto            15054.0
Adventure            13244.0
Scrambler            10037.0
Naked                 9819.0
Adventure Scooter     9064.0
Bobber         

In [7]:

tabla_marca_tipo = pd.pivot_table(
    df,
    index='MARCA',
    columns='TIPO_CORREGIDO',
    aggfunc='size',
    fill_value=0
)

# Mostrar la tabla
print(tabla_marca_tipo)


TIPO_CORREGIDO   Adventure  Adventure Scooter  Adventure Tourer  Bobber  \
MARCA                                                                     
APRILIA                  1                  0                 0       0   
BENELLI                  8                  0                 0       0   
BMW                     11                  0                 0       0   
CF MOTO                  5                  0                 2       0   
DUCATI                   8                  0                 0       0   
HARLEY DAVIDSON          2                  0                 0       0   
HONDA                    7                  2                 0       0   
HUSQVARNA                2                  0                 0       0   
INDIAN                   1                  0                 0       0   
KAWASAKI                 3                  0                 0       0   
KEEWAY                   0                  0                 0       0   
KTM                      

Tenemos demasiadas categorías, difícil visualización y clasificación.

In [ ]:

# Definir el diccionario de agrupación de tipos
mapa_tipos = {
    # Scooter
    'Scooter': 'Scooter',
    'Scooter Eléctrico': 'Scooter',
    'Adventure Scooter': 'Scooter',
    'Maxi Scooter': 'Scooter',
    'Electric': 'Scooter',

    # Adventure
    'Adventure': 'Adventure',
    'Trail': 'Adventure',
    'Adventure Tourer': 'Adventure',
    'Dual Purpose': 'Adventure',

    # Sport
    'Sport': 'Sport',
    'Super Sport': 'Sport',
    'Supermoto': 'Sport',
    'Naked Sport': 'Sport',
    'Sport Touring': 'Sport',
    'Sport Tourer': 'Sport',

    # Custom
    'Scrambler': 'Custom',
    'Bobber': 'Custom',
    'Cafe Racer': 'Custom',
    'Cruiser': 'Custom',

    # Tourer
    'Tourer': 'Tourer',
    'Gran Turismo': 'Tourer',
    'Touring': 'Tourer',

    # Naked
    'Naked': 'Naked',
    'Hypernaked': 'Naked',
    'Hyper-Naked': 'Naked',
    'Classic': 'Naked',

    # Off-road
    'Off-road': 'Off-road',

    # Otro
    'Otro': 'Otro',
    'Otro/no definido': 'Otro',
    'Urban': 'Otro',
    'MiniMOTO': 'Otro'
}

# Crear columna TIPO_SIMPLIFICADO
df['TIPO_SIMPLIFICADO'] = df['TIPO_CORREGIDO'].map(mapa_tipos)

# Tabla dinámica: marcas vs tipos simplificados
tabla_marca_tipo = pd.pivot_table(
    df,
    index='MARCA',
    columns='TIPO_SIMPLIFICADO',
    aggfunc='size',
    fill_value=0
)




In [ ]:
#cuales están en categorias marginal otro 
df.loc[df["TIPO_SIMPLIFICADO"] == "Otro", ["MARCA", "MODELO"]]


,MARCA,MODELO
168,HONDA,MSX 125
173,HONDA,Monkey
174,HONDA,Dax 125
242,INDIAN,Pursuit Dark Horse


In [11]:
df.loc[df['MODELO'].str.contains("Pursuit Dark Horse", case=False), 'TIPO_SIMPLIFICADO'] = 'Tourer'


In [13]:
distribucion_tipo = df['TIPO_SIMPLIFICADO'].value_counts()
porcentaje_tipo = df['TIPO_SIMPLIFICADO'].value_counts(normalize=True) * 100

print("Distribución de Tipos de Motocicletas:")
print(distribucion_tipo)
print("\nPorcentaje de Tipos de Motocicletas:")
print(porcentaje_tipo)

Distribución de Tipos de Motocicletas:
TIPO_SIMPLIFICADO
Scooter      160
Naked        144
Custom        91
Adventure     89
Sport         66
Tourer        36
Off-road       9
Otro           3
Name: count, dtype: int64

Porcentaje de Tipos de Motocicletas:
TIPO_SIMPLIFICADO
Scooter      26.755853
Naked        24.080268
Custom       15.217391
Adventure    14.882943
Sport        11.036789
Tourer        6.020067
Off-road      1.505017
Otro          0.501672
Name: proportion, dtype: float64


In [14]:
modelos_por_marca = df['MARCA'].value_counts()
print("\nNúmero de Modelos por Marca:")
print(modelos_por_marca)


Número de Modelos por Marca:
MARCA
HONDA              44
BMW                41
DUCATI             35
KAWASAKI           33
ROYAL ENFIELD      33
YAMAHA             29
HARLEY DAVIDSON    29
TRIUMPH            29
INDIAN             28
MITT               24
KTM                24
PEUGEOT            23
VESPA              23
ZONTES             23
PIAGGIO            20
APRILIA            20
BENELLI            19
CF MOTO            19
SUZUKI             17
SYM                16
KEEWAY             16
MOTO GUZZI         15
VOGE               12
KYMCO              10
HUSQVARNA          10
MORBIDELLI          6
Name: count, dtype: int64


In [ ]:
df.to_csv('motofit_v1_2.csv', index=True)

NameError: name 'df' is not defined